# Palace Width Sweep: Parallel Simulation

Sweep the microstrip width from 5 to 50 um (5 points), run all simulations
in parallel on GDSFactory+ cloud using the non-blocking API, then compare
S11 and S21 across widths.

**Requirements:**

- IHP PDK: `uv pip install ihp-gdsfactory`
- [GDSFactory+](https://gdsfactory.com) account for cloud simulation

### Define the sweep

In [ ]:
import numpy as np

widths = np.arange(2, 21, 4)

### Build components and configure simulations

In [ ]:
import gdsfactory as gf
from ihp import LAYER, PDK, cells
from gsim.palace import DrivenSim

PDK.activate()

sims = []

for w in widths:
    # Build component
    c = gf.Component()
    r1 = c << cells.straight_metal(length=1000, width=w)

    r = c.get_region(layer=LAYER.TopMetal2drawing)
    r_sized = r.sized(+20000)
    c.add_polygon(r_sized, layer=LAYER.Metal1drawing)
    c.add_ports(r1.ports)

    # Configure simulation
    sim = DrivenSim()
    sim.set_output_dir(f"./palace-sim-w{w:.1f}")
    sim.set_geometry(c)
    sim.set_stack(substrate_thickness=2.0, air_above=300.0)

    for port in c.ports:
        sim.add_port(
            port.name, from_layer="metal1", to_layer="topmetal2", geometry="via"
        )

    sim.set_driven(fmin=1e9, fmax=100e9, num_points=80)
    sim.mesh(preset="default")

    sims.append(sim)

print(f"Configured {len(sims)} simulations")

### Upload and start all jobs (non-blocking)

In [ ]:
# Upload and start all jobs without waiting
job_ids = []
for sim in sims:
    job_id = sim.run(wait=False)
    job_ids.append(job_id)

print(f"Started {len(job_ids)} jobs: {job_ids}")

### Wait for all jobs to complete

In [ ]:
import gsim

# Poll all jobs concurrently, download and parse results
results = gsim.wait_for_results(job_ids)

### Plot S11 and S21 comparison

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(6, 6))

for w, result_files in zip(widths, results):
    csv_path = result_files["port-S.csv"]
    df = pd.read_csv(csv_path)
    df.columns = df.columns.str.strip()
    freq = df["f (GHz)"]

    ax1.plot(freq, df["|S[1][1]| (dB)"], marker=".", label=f"w={w:.1f} um")
    ax2.plot(freq, df["|S[2][1]| (dB)"], marker=".", label=f"w={w:.1f} um")

ax1.set_xlabel("Frequency (GHz)")
ax1.set_ylabel("|S11| (dB)")
ax1.set_title("S11 — Return Loss vs Width")
ax1.legend()
ax1.grid(True)

ax2.set_xlabel("Frequency (GHz)")
ax2.set_ylabel("|S21| (dB)")
ax2.set_title("S21 — Insertion Loss vs Width")
ax2.legend()
ax2.grid(True)

plt.tight_layout()